In [27]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score

In [11]:
turbine= pd.read_csv('/content/gas_turbines.csv')

In [12]:
turbine.head()

AT      AP      AH    AFDP  ...     TEY     CDP      CO     NOX
0  6.8594  1007.9  96.799  3.5000  ...  114.70  10.605  3.1547  82.722
1  6.7850  1008.4  97.118  3.4998  ...  114.72  10.598  3.2363  82.776
2  6.8977  1008.8  95.939  3.4824  ...  114.71  10.601  3.2012  82.468
3  7.0569  1009.2  95.249  3.4805  ...  114.72  10.606  3.1923  82.670
4  7.3978  1009.7  95.150  3.4976  ...  114.72  10.612  3.2484  82.311

[5 rows x 11 columns]

In [13]:
# Normalization function 
def norm_func(i):
     x = (i-i.min())	/	(i.max()	-	i.min())
     return (x)
gasturbine = norm_func(turbine)

In [14]:
x=pd.DataFrame(gasturbine, columns= ['AT','AP','AH','AFDP','GTEP','TIT','TAT','CDP','CO','NOX'])
y=gasturbine.TEY

Tuning hyperparameters- Batch size and epochs

In [15]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [16]:
# create a sequential model
def create_model():
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam()
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

In [17]:
# create regression model
model= KerasRegressor(build_fn=create_model, verbose=0)
# create grid search model
batch_size=[50,100, 200, 400, 800, 1000]
epochs=[10,50,100]
# Make a dictionary of grid search parameters
param_grid= dict(batch_size= batch_size, epochs=epochs)
#build and fit the gridsearchCV
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] batch_size=50, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........... batch_size=50, epochs=10, score=-0.000, total=  13.4s
[CV] batch_size=50, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s


[CV] ........... batch_size=50, epochs=10, score=-0.000, total=   3.8s
[CV] batch_size=50, epochs=10 ........................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.5min remaining:    0.0s


[CV] ........... batch_size=50, epochs=50, score=-0.000, total=  21.5s
[CV] batch_size=50, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.9min remaining:    0.0s


[CV] ........... batch_size=50, epochs=50, score=-0.000, total=  21.6s
[CV] batch_size=50, epochs=100 .......................................
[CV] .......... batch_size=50, epochs=100, score=-0.000, total=  21.6s
[CV] batch_size=50, epochs=100 .......................................
[CV] .......... batch_size=50, epochs=100, score=-0.000, total=  42.0s
[CV] batch_size=50, epochs=100 .......................................
[CV] .......... batch_size=50, epochs=100, score=-0.000, total=  42.0s
[CV] batch_size=50, epochs=100 .......................................
[CV] .......... batch_size=50, epochs=100, score=-0.000, total=  22.1s
[CV] batch_size=50, epochs=100 .......................................
[CV] .......... batch_size=50, epochs=100, score=-0.000, total=  21.7s
[CV] batch_size=100, epochs=10 .......................................
[CV] .......... batch_size=100, epochs=10, score=-0.000, total=   2.2s
[CV] batch_size=100, epochs=10 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 10.6min finished


In [18]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.00012447841727407648, using {'batch_size': 100, 'epochs': 50}
-0.00015420786512549968,2.9207791673154987e-05 with: {'batch_size': 50, 'epochs': 10}
-0.00013427098165266215,5.157183268313777e-05 with: {'batch_size': 50, 'epochs': 50}
-0.0001253275215276517,1.4740803141920446e-05 with: {'batch_size': 50, 'epochs': 100}
-0.00028013489209115504,0.00014149890764774482 with: {'batch_size': 100, 'epochs': 10}
-0.00012447841727407648,2.3849467145762917e-05 with: {'batch_size': 100, 'epochs': 50}
-0.00013431897095870227,2.1997213208121327e-05 with: {'batch_size': 100, 'epochs': 100}
-0.0006596718390937895,0.0003796069237419935 with: {'batch_size': 200, 'epochs': 10}
-0.00013975960755487903,2.2631741797182434e-05 with: {'batch_size': 200, 'epochs': 50}
-0.00013616564829135313,4.23977404176491e-05 with: {'batch_size': 200, 'epochs': 100}
-0.004327453533187508,0.005124344023057221 with: {'batch_size': 400, 'epochs': 10}
-0.00019620152015704663,9.268342339664802e-05 with: {'batch_size': 400

Tuning hyperparameter: Learning rate and drop out rate

In [19]:
from keras.layers import Dropout

# defining model
def create_model_lr_drate(learning_rate,dropout_rate):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam(learning_rate= learning_rate)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_lr_drate, verbose=0, batch_size=100, epochs=100)
# Define grid search model
learning_rate=[0.001, 0.01, 0.1, 0.2, 0.5]
dropout_rate=[0, 0.1, 0.2]
# Make a dictionary of grid search parameters
param_grid= dict(learning_rate= learning_rate, dropout_rate=dropout_rate)
#build and fit the gridsearchCV
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] dropout_rate=0, learning_rate=0.001 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0, learning_rate=0.001, score=-0.000, total=  12.9s
[CV] dropout_rate=0, learning_rate=0.001 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s


[CV]  dropout_rate=0, learning_rate=0.001, score=-0.000, total=  21.9s
[CV] dropout_rate=0, learning_rate=0.001 .............................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.1min remaining:    0.0s


[CV] . dropout_rate=0, learning_rate=0.01, score=-0.000, total=  12.6s
[CV] dropout_rate=0, learning_rate=0.01 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.4min remaining:    0.0s


[CV] . dropout_rate=0, learning_rate=0.01, score=-0.000, total=  21.9s
[CV] dropout_rate=0, learning_rate=0.1 ...............................
[CV] .. dropout_rate=0, learning_rate=0.1, score=-0.060, total=  21.8s
[CV] dropout_rate=0, learning_rate=0.1 ...............................
[CV] .. dropout_rate=0, learning_rate=0.1, score=-0.000, total=  12.3s
[CV] dropout_rate=0, learning_rate=0.1 ...............................
[CV] .. dropout_rate=0, learning_rate=0.1, score=-0.059, total=  12.2s
[CV] dropout_rate=0, learning_rate=0.1 ...............................
[CV] .. dropout_rate=0, learning_rate=0.1, score=-0.043, total=  12.3s
[CV] dropout_rate=0, learning_rate=0.1 ...............................
[CV] .. dropout_rate=0, learning_rate=0.1, score=-0.000, total=  13.0s
[CV] dropout_rate=0, learning_rate=0.2 ...............................
[CV] .. dropout_rate=0, learning_rate=0.2, score=-0.063, total=  21.8s
[CV] dropout_rate=0, learning_rate=0.2 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 22.5min finished


In [20]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.0001633602034416981, using {'dropout_rate': 0, 'learning_rate': 0.01}
-0.00864886201597983,0.017055160047010295 with: {'dropout_rate': 0, 'learning_rate': 0.001}
-0.0001633602034416981,5.975682420386529e-05 with: {'dropout_rate': 0, 'learning_rate': 0.01}
-0.032507395843276755,0.0270287124202494 with: {'dropout_rate': 0, 'learning_rate': 0.1}
-0.04660722240805626,0.01533062170841851 with: {'dropout_rate': 0, 'learning_rate': 0.2}
-0.04454580284655094,0.014232082968759302 with: {'dropout_rate': 0, 'learning_rate': 0.5}
-0.0015551995835267006,0.001249504632926389 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
-0.004532983852550387,0.003105575143569414 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-0.04678572677075863,0.012721727894662152 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-0.047998905181884766,0.015031656140917499 with: {'dropout_rate': 0.1, 'learning_rate': 0.2}
-0.04565753675997257,0.01496866065077261 with: {'dropout_rate': 0.1, 'learning_rate': 0.5}
-0

Tuning hyperparameters: Activation function and Kernal intitilizer

In [21]:
def create_model_af_ker_ini(activation_function,init):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= init, activation='linear')) 
  adam= Adam(learning_rate= 0.001)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_af_ker_ini, verbose=0, batch_size=100, epochs=100)
# Define grid search model
activation_function=['linear', 'relu']
init=['uniform', 'normal', 'zero']
# Make a dictionary of grid search parameters
param_grid= dict(activation_function= activation_function, init=init)
#build and fit the gridsearchCV
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] activation_function=linear, init=uniform ........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation_function=linear, init=uniform, score=-0.000, total=  12.2s
[CV] activation_function=linear, init=uniform ........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


[CV]  activation_function=linear, init=uniform, score=-0.000, total=  12.2s
[CV] activation_function=linear, init=uniform ........................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.3min remaining:    0.0s


[CV]  activation_function=linear, init=normal, score=-0.000, total=  21.8s
[CV] activation_function=linear, init=normal .........................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.6min remaining:    0.0s


[CV]  activation_function=linear, init=normal, score=-0.000, total=  12.1s
[CV] activation_function=linear, init=zero ...........................
[CV]  activation_function=linear, init=zero, score=-0.061, total=  12.2s
[CV] activation_function=linear, init=zero ...........................
[CV]  activation_function=linear, init=zero, score=-0.023, total=  12.2s
[CV] activation_function=linear, init=zero ...........................
[CV]  activation_function=linear, init=zero, score=-0.057, total=  12.1s
[CV] activation_function=linear, init=zero ...........................
[CV]  activation_function=linear, init=zero, score=-0.043, total=  21.7s
[CV] activation_function=linear, init=zero ...........................
[CV]  activation_function=linear, init=zero, score=-0.047, total=  12.1s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=-0.000, total=  21.9s
[CV] activation_function=relu, init=uniform ................

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  8.5min finished


In [22]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.00012277795030968264, using {'activation_function': 'linear', 'init': 'uniform'}
-0.00012277795030968264,1.1558499460509694e-05 with: {'activation_function': 'linear', 'init': 'uniform'}
-0.0001250962566700764,2.1395663066205028e-05 with: {'activation_function': 'linear', 'init': 'normal'}
-0.046145040169358255,0.013113791039457933 with: {'activation_function': 'linear', 'init': 'zero'}
-0.00015165436489041894,3.1511851569997464e-05 with: {'activation_function': 'relu', 'init': 'uniform'}
-0.0001376276239170693,3.9743742302248215e-05 with: {'activation_function': 'relu', 'init': 'normal'}
-0.04637935347855091,0.01311499311844091 with: {'activation_function': 'relu', 'init': 'zero'}


Tuning parameter: Number of neurons in activation layer

In [24]:
def create_model_neuron(neuron1,neuron2, neuron3):
  model= Sequential()
  model.add(Dense(neuron1, input_dim=10, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron2, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron3, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= 'normal', activation='linear')) 
  adam= Adam(learning_rate= 0.001)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_neuron, verbose=0, batch_size=100, epochs=100)
# Define grid search model
neuron1=[4,8,16]
neuron2=[4,8,16]
neuron3=[4,8,16]
# Make a dictionary of grid search parameters
param_grid= dict(neuron1= neuron1, neuron2=neuron2,neuron3=neuron3 )
#build and fit the gridsearchCV
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] neuron1=4, neuron2=4, neuron3=4 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... neuron1=4, neuron2=4, neuron3=4, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=4 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.6s remaining:    0.0s


[CV] .... neuron1=4, neuron2=4, neuron3=4, score=-0.000, total=  11.5s
[CV] neuron1=4, neuron2=4, neuron3=4 .................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.5min remaining:    0.0s


[CV] .... neuron1=4, neuron2=4, neuron3=8, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=8 .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.7min remaining:    0.0s


[CV] .... neuron1=4, neuron2=4, neuron3=8, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=16 ................................
[CV] ... neuron1=4, neuron2=4, neuron3=16, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=16 ................................
[CV] ... neuron1=4, neuron2=4, neuron3=16, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=16 ................................
[CV] ... neuron1=4, neuron2=4, neuron3=16, score=-0.000, total=  11.7s
[CV] neuron1=4, neuron2=4, neuron3=16 ................................
[CV] ... neuron1=4, neuron2=4, neuron3=16, score=-0.000, total=  11.6s
[CV] neuron1=4, neuron2=4, neuron3=16 ................................
[CV] ... neuron1=4, neuron2=4, neuron3=16, score=-0.000, total=  11.4s
[CV] neuron1=4, neuron2=8, neuron3=4 .................................
[CV] .... neuron1=4, neuron2=8, neuron3=4, score=-0.000, total=  11.4s
[CV] neuron1=4, neuron2=8, neuron3=4 .................................
[CV] .

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed: 26.6min finished


In [25]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.0001165233028586954, using {'neuron1': 4, 'neuron2': 8, 'neuron3': 16}
-0.00012443123705452308,1.9664601535166313e-05 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 4}
-0.00012806231679860503,2.197141465349512e-05 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 8}
-0.00011820507352240383,1.7852604405978318e-05 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 16}
-0.00012899670255137608,1.537844726058279e-05 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 4}
-0.00012286052951822058,2.393648831557008e-05 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 8}
-0.0001165233028586954,1.2536508452707207e-05 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 16}
-0.00012218765332363545,2.1693657142859484e-05 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 4}
-0.00012707849818980322,2.281039613943569e-05 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 8}
-0.00012980540777789428,2.9665033311423682e-05 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 16}
-0.00012015225802315399,2.7652540081995042e-05 with: {'n

Model using optimum hyperparameters
Tuning parameter: Number of neurons in activation layer

In [28]:
def create_model_opt():
  model= Sequential()
  model.add(Dense(16, input_dim=10, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= 'normal', activation='linear')) 
  adam= Adam(learning_rate= 0.001)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_opt, verbose=10, batch_size=100, epochs=100)
# Fitting model
estimator= model.fit(x,y)
kfold= KFold(n_splits=10, random_state=1, shuffle=True)
results= cross_val_score(model, x,y, cv= kfold)
print('mse:', results.mean())

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [29]:
print('MSE',results.mean())

MSE -0.00010452642382006161
